Components of the Entire Python3 program:

1. Find the numerical solutions, cn, for any "n" constants (represented by zn) and delta "d."

2. Define the polynomial as a list of coefficients.

3. Find close solutions. When we obtain solutions of the polynomial equations, we are only interested in the previous element of the sequence
   This part of the code returns the "closest" solutions.

4. Setting up a recursion.

5. Checking the solution is close to zero.

6. Checking that all solutions are different.

7. Define A(z1, z2) function. 

8. Define B1 and B2 function. 

9. Define a function for terms inside the summation. 

10. Define a function for the summation. 

11. We check the identity. 

12. Plotting the soltuion. 

In [46]:
#Import libraries for plotting (matplotlib) and advanced mathematical operations (using numpy)
import numpy as np
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')

C:\Users\coe_user\AppData\Local\Temp\ipykernel_20472\1170155103.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-poster')


In [47]:
#Part 1: Find the numerical solutions for any n constants

#FUNCTION DEFINITION: Accepts parameters delta "d" and *z as pointer allowing for any number of z values 
def compute_coefficient(d, *z):
    n = len(z)

    #create an empty list to store the coefficients 
    coeffs = []

    for i in range(n):
        #initalize numerator and denominator for each element in the computation before updating the values in the second loop 
        numerator = 1
        denominator = 1
        
        for j in range(n):

            #compute the numerical solutions for the Bethe equations for the numerator and denominator 
            numerator *= (1 + z[i]*z[j] - d*z[i])
            denominator *= (1 + z[j]*z[i] - d*z[j])

        #append numerical solutions to the result and return as apart of the list
        coeffs.append(-numerator/denominator)

    return coeffs

#FUNCTION CALL : Initialize list z and int d, store returned list and print 
print("\nTest examples for n = 2 and n = 4 cases : \n")

#Works for when 'z' contains two elements!
z2 = [2, 3]
d2 = 4

coeffs2 = compute_coefficient(d2, *z2)
print(coeffs2)

#Check: Product of coefficients (all the elements in the list) should sum to one 
from functools import reduce
import operator
product2 = reduce(operator.mul, coeffs2)
print(product2)

print("\n")

#Works for when 'z' contains four elements!
z4 = [2, 3, 4, 5]
d4 = 6

coeffs4 = compute_coefficient(d4, *z4)
print(coeffs4)

product4 = reduce(operator.mul, coeffs4)
print(product4)


Test examples for n = 2 and n = 4 cases : 

[-0.2, -5.0]
1.0


[-0.004784688995215311, -0.14285714285714285, -3.6666666666666665, -399.0]
1.0


In [48]:

#Part 2: Define the polynomial as the list of coefficients 
def pol_coeffs(l, d, *z):

    coeffs = [1]
    for i in range(l-1):
        coeffs.append(0)
        
    coeffs += [-c for c in compute_coefficient(d, *z)]
    return coeffs

#Test example from BAEsolutionsN2.ipynb 
print("\nTest example from BAE solutions.ipynb : \n")
l1 = 5
d1 = 4
z1 = [2, 3]
print(pol_coeffs(l1, d1, *z1))
print(np.roots(pol_coeffs(l1, d1, *z1)))

print("\n")
l2 = 2
d2 = 0
z2 = [1, 1]
print(pol_coeffs(l2, d2, *z2))
print(np.roots(pol_coeffs(l2, d2, *z2)))


Test example from BAE solutions.ipynb : 

[1, 0, 0, 0, 0, 0.2, 5.0]
[ 1.13816599+0.66355571j  1.13816599-0.66355571j -0.01139753+1.30780953j
 -0.01139753-1.30780953j -1.12676846+0.64380661j -1.12676846-0.64380661j]


[1, 0, 1.0, 1.0]
[ 0.3411639+1.1615414j  0.3411639-1.1615414j -0.6823278+0.j       ]


In [49]:
#Part 3: Finding close solutions

cs_l = 2
cs_z = [1j, -1j]
cs_d = 0.1
cs_s = 1j

def close_sols(l, *zs, d, s):
    sols = np.roots(pol_coeffs(l, *zs, d))
    sols = [x - s for x in sols]
    return min(sols, key=np.abs) + s

print("\nPrinting closed solutions when 'cs_l = 2, cs_z = [1j, -1j], cs_d = 0.1, and cs_s = 1j': \n")
print(close_sols(cs_l, *cs_z, d=cs_d, s=cs_s))


Printing closed solutions when 'cs_l = 2, cs_z = [1j, -1j], cs_d = 0.1, and cs_s = 1j': 

(0.15713911704895112+2.167792837568178j)


In [50]:
#Part 4: Setting up a recursion 

trials = 30

def sol_sys(l, d, *ks, trials):
    sol = np.roots(pol_coeffs(l, 0, 0, 0))
    z = [sol[k] for k in ks]
    for _ in range(trials):
        z = [close_sols(l, *z, d=d, s=z_i) for z_i in z]
    return tuple(z)

ks = [1, 2]
print("\nPrinting the solutions to the system:", sol_sys(5, 0.1, *ks, trials = trials))


Printing the solutions to the system: ((0.7287583876029526-0.4467229293097788j), (-0.15540635445083034+1.1194800146799029j))


In [51]:
#Part 5: Checking The Solution (Showing that all Solutions 'z' are Close to Zero) 
l = 5
d = 0.1
ks = [1, 4]
trials = 30

z = sol_sys(l, d, trials = trials, *ks)
c = compute_coefficient(z[0], z[1], d)  # Replace with the appropriate coefficient function

print("\nChecking solutions are close to zero:", (z[0]**l - c),",", (z[1]**l - c))


Checking solutions are close to zero: [-0.04190611-0.31320768j -1.78033692+1.61317478j] , [ 0.20480212+0.23534168j -1.53362869+2.16172413j]


In [52]:
#Part 6: Checking the Solution (Showing that all Solutions 'z' are Different) 
l = 5
d = .1
trails = 300
all_sol = [sol_sys(l, d, *ks, trials = trials) for k1 in range(l) for k2 in range(l) if k1 != k2]
print("\nPrinting all solutions: ", all_sol)

diff = [abs(x[0]-y[0])+abs(x[1]-y[1]) for x in all_sol for y in all_sol if x != y]
diff.sort()
print("\nPrinting different solutions: ", diff)


Printing all solutions:  [((0.8816575400291621-0.8135306919683141j), (-1.1991792554873144+0.19612519177979773j)), ((0.8816575400291621-0.8135306919683141j), (-1.1991792554873144+0.19612519177979773j)), ((0.8816575400291621-0.8135306919683141j), (-1.1991792554873144+0.19612519177979773j)), ((0.8816575400291621-0.8135306919683141j), (-1.1991792554873144+0.19612519177979773j)), ((0.8816575400291621-0.8135306919683141j), (-1.1991792554873144+0.19612519177979773j)), ((0.8816575400291621-0.8135306919683141j), (-1.1991792554873144+0.19612519177979773j)), ((0.8816575400291621-0.8135306919683141j), (-1.1991792554873144+0.19612519177979773j)), ((0.8816575400291621-0.8135306919683141j), (-1.1991792554873144+0.19612519177979773j)), ((0.8816575400291621-0.8135306919683141j), (-1.1991792554873144+0.19612519177979773j)), ((0.8816575400291621-0.8135306919683141j), (-1.1991792554873144+0.19612519177979773j)), ((0.8816575400291621-0.8135306919683141j), (-1.1991792554873144+0.19612519177979773j)), ((0.8

In [53]:
#Part 7: Checking the Initial Conditions 

def A(z1, z2, d):
    return (-1)* (1 + z1*z2 - d*z2)/(1 + z1 *z2 - d*z1)

print("\nChecking initial condition A: ", A(1,2,4)) #expected -5.0



Checking initial condition A:  -5.0


In [54]:
#Part 8: Define B1 B2 Function


In [55]:
#Part 9: Define a Function for terms inside the Summation

In [56]:
#Part 10: Define a Function for the Summation

In [57]:
#Part 11: We Check the Identity 

In [58]:
#Part 12: Plotting the Solution